In [1]:
from openai import AzureOpenAI
import gradio as gr
import json

c:\Users\nikunj.bedia\OneDrive - Providence St. Joseph Health\Hackathon\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

True

In [3]:
system_prompt="""
System settings:
Tool use: enabled.

Instructions:
- You are an AI medical assistant responsible for helping patients complete an intake form for back pain through voice interaction.
- Use the tools to fetch questions to pose.
- Start by greeting the user and calling the get_next_question tool without a currentPageId to get the first question from the root page.
- After presenting each question to the user, wait for their response.
- Use the get_next_question tool again with the current pageId to fetch the next question.
- If a question has multiple choice options, present them clearly to the user and wait for their selection.
- If the tool returns END-OF-FORM-QUESTIONS, conclude the intake process and thank the user.
- Speak clearly and professionally, as if you were a medical professional conducting an intake interview.
- If the user asks for clarification or has questions, respond helpfully while staying focused on completing the intake form.
- Use other available tools as needed to enhance the interaction or gather additional information.
- Do not attempt to process or interpret the user's responses within the tool call. Simply present the questions and options as provided by the tool.

Personality:
- Be professional, patient, and empathetic
- Speak clearly and at a moderate pace
- Use a calm and reassuring tone

Remember: Your primary goal is to guide the user through the intake form for back pain by presenting questions. Do not make assumptions about the user's condition or provide medical advice.
"""

In [4]:
jsonraw=open("Async Care - Back Pain copy.json")
asyncCareBackPainData=json.load(jsonraw)

In [5]:
def get_next_question(currentPageId=None, selectedOption=None):
    jsonData = asyncCareBackPainData  # assuming asyncCareBackPainData is already defined
    nextPageId = ""

    if currentPageId is None:
        nextPageId = jsonData.get("algorithmRootPageId","")
    elif selectedOption and "link" in selectedOption:
        nextPageId = jsonData["nameToPageId"].get(selectedOption["link"], "")
    elif jsonData["pages"].get(currentPageId, {}).get("next_link"):
        nextLink = jsonData["pages"][currentPageId]["next_link"]
        if nextLink == "END":
            return {"tool_response": "END-OF-FORM-QUESTIONS"}
        if isinstance(nextLink, str):
            nextPageId = jsonData["nameToPageId"].get(nextLink, "")
    else:
        nextPageId = currentPageId

    if not nextPageId:
        return {"error": "No next question found"}

    nextPage = jsonData["pages"].get(nextPageId, {})

    return {
        "pageId": nextPageId,
        "questions": nextPage.get("questions", [])
        # "content": nextPage.get("content", None)  # Uncomment if needed
    }


In [6]:
tools = [
    {
        "type": "function",
        "function":
        {
          "name": 'get_next_question',
          "description":'Fetches the next set of questions to ask the user for their medical concerns.',
          "parameters": 
          {
            "type": 'object',
            "properties": 
            {
              "currentPageId": 
              {
                "type": 'string',
                "description":
                  'The ID of the current page. If not provided, the root page will be used.',
              },
              "selectedOption": 
              {
                "type": 'object',
                "properties": 
                {
                  "id": { "type": 'string' },
                  "text": { "type": 'string' },
                  "link": { "type": 'string' },
                },
                "description": 'The option selected by the user (if any)',
              }
            }
          }
        }
    }  
]

In [7]:
print(os.getenv('azure_endpoint'))

https://oai-aurigenai-dev-02.openai.azure.com/


In [8]:
client = AzureOpenAI(
  azure_endpoint = os.getenv('azure_endpoint'), 
  api_key=os.getenv('api_key'), 
  api_version="2024-02-15-preview"
)

  
deployment_name = os.getenv('deployment_name')  # Replace with your model deployment name

def predict(message, history):
    history_openai_format = []
    history_openai_format.append({
        "role": "system",
        "content": system_prompt
    })
    for msg in history:
        history_openai_format.append(msg)
    
    history_openai_format.append({
        "role": "user",
        "content": message
    })

    # print(history_openai_format)
  
    response = client.chat.completions.create(model=deployment_name,
    messages= history_openai_format,
    temperature=1.0,
    stream=False,
    tools=tools)

    print(response)
    
    return response.choices[0].message.content

    # partial_message = ""
    # for chunk in response:
    #     if chunk.choices[0].delta.content is not None:
    #           partial_message = partial_message + chunk.choices[0].delta.content
    #           yield partial_message

gr.ChatInterface(predict, type="messages").launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='chatcmpl-AKvwpfcYvzgvuC68ysO8OhYJYTFEe', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm here to help you complete an intake form for your back pain. Let's get started with the first question.\n\nJust a moment, please.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lifpL68a23Kh8AOkgY9goHWD', function=Function(arguments='{}', name='get_next_question'), type='function')]), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1729552675, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_67802d9a6d', usage=CompletionUsage(completion_tokens=42, prompt_tokens=399, total_tokens=441, completion_tokens_detail